<a href="https://colab.research.google.com/github/dotsnangles/Transformers-Text-Classification/blob/main/%EB%89%B4%EC%8A%A4_%ED%86%A0%ED%94%BD_%EB%B6%84%EB%A5%98_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -q folium
!pip install -q transformers
!pip install -q datasets
!pip install -q huggingface_hub

In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import datasets
import pandas as pd
from huggingface_hub import notebook_login

In [12]:
from sklearn.metrics import accuracy_score, f1_score

In [29]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [13]:
num_labels = 7

In [14]:
model_ckpt = 'klue/roberta-small'
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels,)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classif

In [15]:
data = datasets.load_dataset('csv', data_files='/content/train_data.csv')['train']
data = data.train_test_split(test_size=0.2)
data = data.remove_columns('index')

Using custom data configuration default-76eab955a46b3527
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-76eab955a46b3527/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
data

DatasetDict({
    train: Dataset({
        features: ['title', 'topic_idx'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['title', 'topic_idx'],
        num_rows: 9131
    })
})

In [17]:
def tokenize(batch):
    return tokenizer(batch["title"], padding=True, truncation=True)

In [18]:
encoded_data = data.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['title', 'topic_idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['title', 'topic_idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9131
    })
})

In [20]:
encoded_data = encoded_data.rename_column('topic_idx', 'label')

In [21]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['title', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 36523
    })
    test: Dataset({
        features: ['title', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9131
    })
})

In [22]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [23]:
batch_size = 64
logging_steps = len(encoded_data['train']) // batch_size
model_name = f"{model_ckpt}-finetuned-klue-roberta-tc"
training_args = TrainingArguments(output_dir=model_name,
                                    num_train_epochs=2,
                                    learning_rate=2e-5,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    weight_decay=0.01,
                                    evaluation_strategy="epoch",
                                    disable_tqdm=False,
                                    logging_steps=logging_steps,
                                    push_to_hub=True,
                                    log_level="error")

In [30]:
trainer = Trainer(model=model, args=training_args,
                    compute_metrics=compute_metrics,
                    train_dataset=encoded_data['train'],
                    eval_dataset=encoded_data['test'],
                    tokenizer=tokenizer)
trainer.train()

Cloning https://huggingface.co/dotsnangles/roberta-small-finetuned-klue-roberta-tc into local empty directory.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.496700,0.334563,0.886431,0.885905
2,0.298800,0.327523,0.886978,0.886006


TrainOutput(global_step=1142, training_loss=0.39764305756096247, metrics={'train_runtime': 142.2644, 'train_samples_per_second': 513.452, 'train_steps_per_second': 8.027, 'total_flos': 548115656028972.0, 'train_loss': 0.39764305756096247, 'epoch': 2.0})

In [31]:
data_test = datasets.load_dataset('csv', data_files='/content/test_data.csv')['train']

Using custom data configuration default-0423ff50118ab744


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0423ff50118ab744/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
data_test = data_test.remove_columns('index')

In [36]:
encoded_test = data_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [37]:
encoded_test

Dataset({
    features: ['title', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9131
})

In [46]:
preds_output = trainer.predict(encoded_data['test'])

In [47]:
preds_output.metrics

{'test_accuracy': 0.88697842514511,
 'test_f1': 0.8860059287326812,
 'test_loss': 0.3275226056575775,
 'test_runtime': 5.3994,
 'test_samples_per_second': 1691.099,
 'test_steps_per_second': 26.484}